In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision.transforms as transforms
from datasets.TestDataset import TestDataset
import matplotlib.pyplot as plt
from examples.mask_to_submission import *
import random
import torchvision.models.segmentation as models

In [3]:
# set seeds as function
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


# set seeds
set_seed()

In [4]:
# select checkpoint
MODEL = "models/unet.pt"

In [14]:
# load checkpoint
checkpoint = torch.load(MODEL)
# create model
# model = torch.hub.load(
#    "milesial/Pytorch-UNet", "unet_carvana", pretrained=False, scale=0.5
# )
model = torch.hub.load(
    "mateuszbuda/brain-segmentation-pytorch",
    "unet",
    in_channels=3,
    out_channels=1,
    init_features=32,
    pretrained=False,
)

# load model weights
model.load_state_dict(checkpoint)

Using cache found in /home/stef/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


<All keys matched successfully>

In [11]:
# path to the test folder
test_folder = "datasets/test/"

# define transformations
transform = transforms.Compose([transforms.ToTensor()])

# create test dataset
test_dataset = TestDataset(test_folder, transform=transform)

['datasets/test/test_1/test_1.png', 'datasets/test/test_2/test_2.png', 'datasets/test/test_3/test_3.png', 'datasets/test/test_4/test_4.png', 'datasets/test/test_5/test_5.png', 'datasets/test/test_6/test_6.png', 'datasets/test/test_7/test_7.png', 'datasets/test/test_8/test_8.png', 'datasets/test/test_9/test_9.png', 'datasets/test/test_10/test_10.png', 'datasets/test/test_11/test_11.png', 'datasets/test/test_12/test_12.png', 'datasets/test/test_13/test_13.png', 'datasets/test/test_14/test_14.png', 'datasets/test/test_15/test_15.png', 'datasets/test/test_16/test_16.png', 'datasets/test/test_17/test_17.png', 'datasets/test/test_18/test_18.png', 'datasets/test/test_19/test_19.png', 'datasets/test/test_20/test_20.png', 'datasets/test/test_21/test_21.png', 'datasets/test/test_22/test_22.png', 'datasets/test/test_23/test_23.png', 'datasets/test/test_24/test_24.png', 'datasets/test/test_25/test_25.png', 'datasets/test/test_26/test_26.png', 'datasets/test/test_27/test_27.png', 'datasets/test/tes

In [7]:
# save predictions
prediction_filenames = []
for i in range(len(test_dataset)):
    # get image
    image = test_dataset[i]
    # create prediction
    prediction = model(image.unsqueeze(0))
    # for milestial (remove for mateuszbuda)
    prediction = prediction[:, [0], :, :]
    # threshold prediction
    prediction = (prediction > 0.5).float()
    # save prediction
    prediction_filename = "predictions/prediction_" + str(i + 1) + ".png"
    prediction_filenames.append(prediction_filename)
    plt.imsave(prediction_filename, prediction.squeeze().detach().numpy(), cmap="gray")

In [14]:
# create submission
masks_to_submission("submission.csv", *prediction_filenames)